In [2]:
# Import necessary libraries
import pandas as pd
import requests
import json

# Import necessary libraries
import json

# Load the configuration file
with open('config.json') as config_file:
    config = json.load(config_file)

# Get the API key from the configuration file
GOOGLE_FONTS_API_KEY = config['GOOGLE_FONTS_API_KEY']

# Set the API endpoint for the Google Fonts API
GOOGLE_FONTS_API_ENDPOINT = 'https://www.googleapis.com/webfonts/v1/webfonts'

# Set the parameters for the API request
params = {
    'key': GOOGLE_FONTS_API_KEY,
    'sort': 'popularity'
}

# Make the API request to get the list of fonts
response = requests.get(GOOGLE_FONTS_API_ENDPOINT, params=params)

# Parse the response as JSON
fonts = json.loads(response.text)

# Create a dictionary for storing the fonts in the database
database = {}

# Loop through each font in the response
for font in fonts["items"]:
    # Get the font's family name
    family = font['family']
    
    # Get the font's category
    category = font['category']
    
    # Store the font in the database
    database[family] = {
        'family': family,
        'category': category,
        'variants': font['variants'],
        'subsets': font['subsets'],
        'version': font['version'],
        'lastModified': font['lastModified']
    }

# Function for generating a dataset of fonts
def generate_font_dataset(num_fonts, categories=None, subsets=None, versions=None):
    # Initialize an empty list for storing the fonts in the dataset
    font_dataset = []
    
    # Loop through the database of fonts
    for font in database.values():
        # Check if the font's category matches the specified categories (if provided)
        if categories and font['category'] not in categories:
            continue
        
        # Check if the font's subsets match the specified subsets (if provided)
        if subsets and not set(font['subsets']).issubset(subsets):
            continue
        
        # Check if the font's version matches the specified versions (if provided)
        if versions and font['version'] not in versions:
            continue
        
        # Add the font to the dataset
        font_dataset.append(font)
        
        # Stop adding fonts to the dataset if the desired number of fonts has been reached
        if len(font_dataset) >= num_fonts:
            break
    
    # Return the generated dataset of fonts
    return font_dataset

# Example usage: generate a dataset of 10 sans-serif fonts that support the Latin and Cyrillic character sets
font_dataset = generate_font_dataset(10, categories=['sans-serif'], subsets=['latin', 'cyrillic'])

# Print the names of the fonts in the dataset
for font in font_dataset:
    print(font['family'])

# Create a DataFrame object from the font database
font_database_df = pd.DataFrame.from_dict(database, orient='index')
font_database_df

Abel
Acme
Orbitron
Michroma
Outfit
Actor
Amaranth
Allerta Stencil
Quantico
Electrolize


,family,category,variants,subsets,version,lastModified
Roboto,Roboto,sans-serif,"[100, 100italic, 300, 300italic, regular, ital...","[cyrillic, cyrillic-ext, greek, greek-ext, lat...",v30,2022-09-22
Open Sans,Open Sans,sans-serif,"[300, regular, 500, 600, 700, 800, 300italic, ...","[cyrillic, cyrillic-ext, greek, greek-ext, heb...",v34,2022-09-22
Noto Sans JP,Noto Sans JP,sans-serif,"[100, 300, regular, 500, 700, 900]","[japanese, latin]",v42,2022-09-27
Montserrat,Montserrat,sans-serif,"[100, 200, 300, regular, 500, 600, 700, 800, 9...","[cyrillic, cyrillic-ext, latin, latin-ext, vie...",v25,2022-09-22
Lato,Lato,sans-serif,"[100, 100italic, 300, 300italic, regular, ital...","[latin, latin-ext]",v23,2022-09-22
...,...,...,...,...,...,...
Rubik Storm,Rubik Storm,display,[regular],"[cyrillic, cyrillic-ext, hebrew, latin, latin-...",v1,2022-12-08
Noto Serif NP Hmong,Noto Serif NP Hmong,serif,"[regular, 500, 600, 700]","[latin, nyiakeng-puachue-hmong]",v1,2022-12-08
Rubik Spray Paint,Rubik Spray Paint,display,[regular],"[cyrillic, cyrillic-ext, hebrew, latin, latin-...",v1,2022-12-08
Chivo Mono,Chivo Mono,monospace,"[100, 200, 300, regular, 500, 600, 700, 800, 9...","[latin, latin-ext, vietnamese]",v4,2022-12-08
